In [61]:
import glob
import os
from dotenv import load_dotenv
import boto3
import time
from datetime import datetime, timedelta
import pandas as pd
import csv
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import itertools

In [62]:
load_dotenv()
# First Screen Data - Scraper
dest_bucket = os.getenv("DEST_BUCKET")
date = (datetime.today() - timedelta(0)).strftime('%Y%m%d')
year = (datetime.today() - timedelta(0)).strftime('%Y-%m-%d').split('-')[0]
month = (datetime.today() - timedelta(0)).strftime('%Y-%m-%d').split('-')[1]
day = (datetime.today() - timedelta(0)).strftime('%Y-%m-%d').split('-')[2]
landing_filepath = os.getenv("LANDING_FILE") + date + ".csv"
s3 = boto3.resource('s3')
s3client = boto3.client('s3', region_name='us-east-2')

## Adapter Layer

In [63]:
# Adapter Layer
def driver_with_options():
    options = Options()
    # options.headless = True
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def find_total_pages(driver, xpath_location):
    total_pages = [sub for sub in driver.find_element(By.XPATH, xpath_location).text.split() if sub.isdigit()][1]
    return total_pages
    
def find_total_rows_by_page(driver, xpath_location):
    total_rows = driver.find_elements(By.XPATH, xpath_location)
    return len(total_rows)
    
def click_and_wait(driver, xpath_location):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_location))).click()
    time.sleep(7)
    return driver

def click_on(driver, xpath_location):
    driver.find_element(By.XPATH, xpath_location).click()
    time.sleep(3)


## Application Layer

In [64]:
# Application Layer
driver = driver_with_options()
driver.get(os.getenv('URL'))
time.sleep(4)


In [5]:
# Clicking on the the License Type Radio Button
license_type_radio_btn = click_and_wait(driver, xpath_location='/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-radio-group/slot/fieldset/div[1]/div[2]/div[2]/label[1]/span[1]')

# Clicking on the Dropdown  - License Type
driver.find_element(By.XPATH,
                    '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-select[1]/slot/c-combobox/div/div/div[2]/div[1]/div/input').click()

time.sleep(3)

# Click on the drop down combobox
print('clicked on combobox')


# driver.find_element(By.XPATH,
#                     '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-select[1]/slot/c-combobox/div/div/div[2]/div[2]/div/ul/li[2]/div/span/span').click()
# time.sleep(4)

clicked on combobox


In [36]:
# time.sleep(2)
# driver.find_element(By.XPATH,
#                     '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-select[1]/slot/c-combobox/div/div/div[2]/div[2]/div/ul/li[2]/div/span/span').click()
# time.sleep(4)


In [37]:
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,
#                                                             '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-block/ul/li/section/div[2]/slot/vlocity_ins-omniscript-ip-action/slot/div/c-button/button'))).click()
# time.sleep(20)

In [6]:
total_pages = find_total_pages(driver=driver, xpath_location='/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-text-block[6]/slot/div/c-omniscript-formatted-rich-text/lightning-formatted-rich-text/span/div/p')
print(total_pages)
# print("Total Number of Rows are {} in Page {} : ".format(len(total_rows), page))


13


In [11]:
def create_list_of_columns(list_of_xpaths, new_column_list):
    for elem in range(0, len(list_of_xpaths)):
        # print(elem)
        new_column_list[elem] = []
    return new_column_list


def creating_df(df_name, column_list_data):
    for elem in range(0, len(column_list_data)):
        df_name[elem] = column_list_data[elem]
    return df_name


def insert_data(file_name, data):
    with open(file_name, 'w+') as csvfile:
        writer = csv.writer(csvfile)
        for index, row in data.iterrows():
            writer.writerow(row)
            
def extract_text_from_xpath_list(xpath_list, extracted_list):
    for elem in range(0, len(xpath_list)):
        for col_name in driver.find_elements(By.XPATH, xpath_list[elem]):
            extracted_list[elem].append(col_name.text)
    return extracted_list

In [12]:
# def scrape_data_by_page(list_of_xpath, list_of_columns, df_col_list, total_rows, page_counter, output_file):
def scrape_data_by_page(list_of_xpath, list_of_columns, total_rows, page_counter):
    for page in range(int(page_counter)):
        for row in range(0, len(total_rows)):
            rec = extract_text_from_xpath_list(xpath_list = list_of_xpath, extracted_list = list_of_columns)
            print(rec)
            return rec

In [60]:
all_data = pd.DataFrame()
for p in total_pages:
    row_count = driver.find_elements(By.XPATH,
                                    "/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-custom-lwc[3]/slot/c-cf-mtxlpi_fc_-c-c-dcomprehensive-application-search_-child/div/vlocity_ins-flex-card-state")
    print("Total Number of Rows are {} in Page {} : ".format(len(row_count), p))
    
    row_list = []
    li_type = []
    app_status = []
    busi_name = []
    busi_addr = []
    for row in range(1,len(row_count)):
        xli = [
    '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-custom-lwc[3]/slot/c-cf-mtxlpi_fc_-c-c-dcomprehensive-application-search_-child/div/vlocity_ins-flex-card-state[{}]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div[1]/vlocity_ins-output-field/div/lightning-formatted-rich-text/span/div'.format(row),
    '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-custom-lwc[3]/slot/c-cf-mtxlpi_fc_-c-c-dcomprehensive-application-search_-child/div/vlocity_ins-flex-card-state[{}]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div[2]/vlocity_ins-output-field/div/lightning-formatted-rich-text/span'.format(row),
    '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-custom-lwc[3]/slot/c-cf-mtxlpi_fc_-c-c-dcomprehensive-application-search_-child/div/vlocity_ins-flex-card-state[{}]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div[3]/vlocity_ins-output-field/div/lightning-formatted-rich-text/span/div'.format(row),
    '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-custom-lwc[3]/slot/c-cf-mtxlpi_fc_-c-c-dcomprehensive-application-search_-child/div/vlocity_ins-flex-card-state[{}]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div[4]/vlocity_ins-output-field/div/lightning-formatted-rich-text/span'.format(row)]
        column_list = create_list_of_columns(list_of_xpaths=xli,
                                     new_column_list=['license_type', 'application_status', 'business_name',
                                                      'business_address'])
        rec = scrape_data_by_page(
            list_of_xpath=xli, 
            list_of_columns=column_list,
            total_rows=row_count,
            page_counter=total_pages)
        print(rec[0])
        li_type.append(rec[0])
        app_status.append(rec[1])
        busi_name.append(rec[2])
        busi_addr.append(rec[3])
    page_df = pd.DataFrame(list(zip(li_type, app_status, busi_name, busi_addr)), columns=['license_type', 'application_status', 'business_name', 'business_address'])
#     print(page_df)
    all_data = pd.concat([all_data,page_df], axis=0)
    print("DataFrame: ", p)
    # Next Button Click
    time.sleep(7)
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,
                                                                        '/html/body/div[3]/div[3]/div/div[2]/div/div/c-cf-mtxlpi_fc_comprehensive-license-search-container/div/vlocity_ins-flex-card-state[2]/div/slot/div/div/vlocity_ins-block/div/div/div/slot/div/div/vlocity_ins-custom-lwc-wrapper/slot/c-c-c-d-comprehensive-license-search-c-c-d-english/div/article/div[1]/vlocity_ins-omniscript-step/div[3]/slot/vlocity_ins-omniscript-ip-action[3]/slot/div/c-button/button'))).click()
    except ValueError:
        break
    time.sleep(10)

# print(all_data)

Total Number of Rows are 57 in Page 1 : 
[['Cannabis Manufacturer'], ['Approved'], ['Hype Boyz Genetics, LLC'], ['']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['Northwest Confections New Mexico LLC'], ['6919 Montgomery Blvd NE Albuquerque NM 87109']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['CannaBliss LLC'], ['2883 NM 14 Madrid NM 87010']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['Flynn Family Farm LLC'], ['']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['Leaf Love Organic'], ['']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Not Approved'], ['Fireflower Investments'], ['']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['Dulce Cannabis LLC'], ['2000 Carlisle Blvd NE B Unit B Albuquerque NM 87110']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Approved'], ['Score 420'], ['']]
['Cannabis Manufacturer']
[['Cannabis Manufacturer'], ['Appr

KeyboardInterrupt: 

In [ ]:
dest_bucket = os.getenv("DEST_BUCKET")
destination_file = os.getenv("DEST_FILE_LOCATION") + date + os.getenv("DEST_FILE")
csvfile = []
for f in glob.glob('*.csv'):
    if f.startswith('first_screen_data_'):
        csvfile.append(f)

for datafile in csvfile:
    print(datafile)
    #    s3client.upload_file(datafile, Bucket=dest_bucket, Key=destination_file)
    print('File uploaded on S3.')

driver.close()
print("Done.")